[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_06/notebooks/NB13_Robo_Advisor.ipynb)

# NB13: Robo-Advisor Simulation

**Topic:** 6.2 - AI and Digital Finance: Automated Wealth Management

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Robo-Advisor Mechanics**: Learn how automated investment platforms work
2. **Implement Portfolio Optimization**: Build mean-variance optimization from scratch
3. **Risk Profiling**: Create questionnaire-based investor classification
4. **Automated Rebalancing**: Implement threshold and calendar-based rebalancing strategies
5. **Tax-Loss Harvesting**: Understand the basics of tax-efficient investing
6. **Compare Approaches**: Evaluate robo vs traditional advisors

In [ ]:
!pip install -q numpy matplotlib

## Section 1: Setup

We'll build a complete robo-advisor simulation using only Python standard libraries and numpy/matplotlib.

The code below loads the tools our robo-advisor simulation needs. You don't need to understand the code --- just run it and focus on the outputs.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print("\nWe're ready to build a robo-advisor from scratch.")

## Section 2: What is a Robo-Advisor?

### Industry Context

A **robo-advisor** is an automated platform that provides digital financial advice and investment management with minimal human intervention. They use algorithms to:

1. **Assess Risk Tolerance**: Through questionnaires
2. **Build Portfolios**: Using Modern Portfolio Theory (MPT)
3. **Rebalance Automatically**: Maintain target allocations
4. **Optimize Taxes**: Through tax-loss harvesting

### Key Players in the Market

| Platform | AUM (2024) | Min Investment | Annual Fee |
|----------|------------|----------------|------------|
| Betterment | $40B+ | $0 | 0.25% |
| Wealthfront | $50B+ | $500 | 0.25% |
| Schwab Intelligent | $80B+ | $5,000 | 0.00% |
| Vanguard Digital | $300B+ | $3,000 | 0.15% |
| BlackRock (Aladdin) | Institutional | - | - |

> **Note on Schwab's 0.00% fee:** While Schwab charges no advisory fee, they earn revenue through cash allocation requirements (holding a portion of your portfolio in low-yield cash) and other means. No service is truly free.

### How They Work

In [ ]:
def explain_robo_advisor_workflow():
    """
    Display the robo-advisor workflow.
    """
    workflow = """
    ROBO-ADVISOR WORKFLOW
    =====================
    
    Step 1: ONBOARDING
    +-----------------+
    | Risk Assessment |  --> Questionnaire (5-15 questions)
    | Goal Setting    |  --> Time horizon, target amount
    | Account Setup   |  --> Link bank, KYC verification
    +-----------------+
            |
            v
    Step 2: PORTFOLIO CONSTRUCTION
    +------------------------+
    | Risk Profile Mapping   |  --> Conservative to Aggressive
    | Asset Allocation       |  --> Stocks, Bonds, Alternatives
    | ETF Selection          |  --> Low-cost index funds
    +------------------------+
            |
            v
    Step 3: ONGOING MANAGEMENT
    +------------------------+
    | Automatic Deposits     |  --> Dollar-cost averaging
    | Rebalancing            |  --> Threshold or calendar-based
    | Tax-Loss Harvesting    |  --> Offset gains with losses
    | Performance Reporting  |  --> Dashboard and alerts
    +------------------------+
    
    Key Advantages:
    - Low fees (0.15-0.50% vs 1-2% traditional)
    - No minimum or low minimum investment
    - 24/7 access and automated optimization
    - Emotion-free, disciplined investing
    """
    print(workflow)

explain_robo_advisor_workflow()

## Section 3: Risk Profiling Questionnaire

The first step in any robo-advisor is assessing the investor's risk tolerance. Let's build a questionnaire-based risk profiler.

**Just run the next cell -- no programming knowledge needed!** This section simulates how a robo-advisor asks about your risk tolerance and translates your answers into an investment strategy. The code sets up the questionnaire logic that real robo-advisors use.

In [ ]:
@dataclass
class RiskQuestion:
    """A single risk assessment question."""
    question: str
    options: List[str]
    scores: List[int]  # Score for each option (higher = more risk tolerant)

class RiskProfiler:
    """
    Risk profiling questionnaire system.
    
    Assesses investor risk tolerance through standardized questions
    and maps to portfolio recommendations.
    """
    
    def __init__(self):
        self.questions = self._create_questions()
        self.responses = []
        
    def _create_questions(self) -> List[RiskQuestion]:
        """Create the risk assessment questionnaire."""
        return [
            RiskQuestion(
                question="What is your investment time horizon?",
                options=[
                    "Less than 1 year",
                    "1-3 years",
                    "3-7 years",
                    "7-15 years",
                    "More than 15 years"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
            RiskQuestion(
                question="If your portfolio dropped 20% in one month, what would you do?",
                options=[
                    "Sell everything immediately",
                    "Sell some investments",
                    "Do nothing and wait",
                    "Buy more at lower prices",
                    "Significantly increase investment"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
            RiskQuestion(
                question="What is your primary investment goal?",
                options=[
                    "Preserve capital at all costs",
                    "Generate steady income",
                    "Balance growth and income",
                    "Focus on long-term growth",
                    "Maximize aggressive growth"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
            RiskQuestion(
                question="How much investment experience do you have?",
                options=[
                    "None - I'm new to investing",
                    "Limited - savings accounts only",
                    "Some - mutual funds/ETFs",
                    "Good - stocks and bonds",
                    "Extensive - options, alternatives"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
            RiskQuestion(
                question="What percentage of your total savings is this investment?",
                options=[
                    "More than 75%",
                    "50-75%",
                    "25-50%",
                    "10-25%",
                    "Less than 10%"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
            RiskQuestion(
                question="How stable is your current income?",
                options=[
                    "Very unstable/unemployed",
                    "Somewhat unstable",
                    "Moderately stable",
                    "Stable employment",
                    "Very stable with multiple income sources"
                ],
                scores=[1, 2, 3, 4, 5]
            ),
        ]
    
    def calculate_risk_score(self, responses: List[int]) -> Dict:
        """
        Calculate risk score from questionnaire responses.
        
        Args:
            responses: List of selected option indices (0-based)
            
        Returns:
            Dictionary with risk score and profile
        """
        if len(responses) != len(self.questions):
            raise ValueError(f"Expected {len(self.questions)} responses, got {len(responses)}")
        
        total_score = 0
        max_score = 0
        
        for i, (question, response) in enumerate(zip(self.questions, responses)):
            total_score += question.scores[response]
            max_score += max(question.scores)
        
        # Normalize to 0-100 scale
        normalized_score = (total_score / max_score) * 100
        
        # Map to risk profile
        if normalized_score < 25:
            profile = "Conservative"
            description = "Capital preservation is your priority. You prefer stable, low-risk investments."
        elif normalized_score < 45:
            profile = "Moderately Conservative"
            description = "You seek modest growth with limited volatility."
        elif normalized_score < 65:
            profile = "Moderate"
            description = "You balance growth potential with risk management."
        elif normalized_score < 85:
            profile = "Moderately Aggressive"
            description = "You prioritize growth and can tolerate significant short-term volatility."
        else:
            profile = "Aggressive"
            description = "You seek maximum growth and are comfortable with high volatility."
        
        return {
            'raw_score': total_score,
            'max_score': max_score,
            'normalized_score': normalized_score,
            'profile': profile,
            'description': description
        }
    
    def display_questionnaire(self):
        """Display all questions for reference."""
        print("\n" + "="*70)
        print("RISK ASSESSMENT QUESTIONNAIRE")
        print("="*70)
        
        for i, q in enumerate(self.questions, 1):
            print(f"\nQ{i}: {q.question}")
            for j, option in enumerate(q.options):
                print(f"    {j+1}. {option} (score: {q.scores[j]})")

# Create and display the questionnaire
profiler = RiskProfiler()
profiler.display_questionnaire()

In [ ]:
# Simulate different investor profiles
print("\n" + "="*70)
print("SIMULATED INVESTOR PROFILES")
print("="*70)

# Conservative investor (Sarah, 62, retiring soon)
conservative_responses = [0, 0, 0, 1, 0, 2]  # All low-risk answers
conservative_result = profiler.calculate_risk_score(conservative_responses)

# Moderate investor (Michael, 42, saving for retirement)
moderate_responses = [2, 2, 2, 2, 2, 3]  # Middle-of-the-road
moderate_result = profiler.calculate_risk_score(moderate_responses)

# Aggressive investor (Alex, 28, long time horizon)
aggressive_responses = [4, 4, 4, 3, 4, 4]  # High-risk tolerance
aggressive_result = profiler.calculate_risk_score(aggressive_responses)

# Display results
investors = [
    ("Sarah (62, pre-retirement)", conservative_result),
    ("Michael (42, mid-career)", moderate_result),
    ("Alex (28, early career)", aggressive_result)
]

for name, result in investors:
    print(f"\n{name}")
    print("-" * 40)
    print(f"  Risk Score: {result['normalized_score']:.1f}/100")
    print(f"  Profile: {result['profile']}")
    print(f"  {result['description']}")

In [ ]:
# Visualize risk profiles
fig, ax = plt.subplots(figsize=(10, 6))

profiles = ['Conservative', 'Mod. Conservative', 'Moderate', 'Mod. Aggressive', 'Aggressive']
score_ranges = [(0, 25), (25, 45), (45, 65), (65, 85), (85, 100)]
colors = ['#2E7D32', '#66BB6A', '#FFC107', '#FF7043', '#D32F2F']

# Create horizontal bars for score ranges
for i, (profile, (start, end), color) in enumerate(zip(profiles, score_ranges, colors)):
    ax.barh(0, end - start, left=start, height=0.5, color=color, alpha=0.7, label=profile)

# Mark investor positions
investor_scores = [
    (conservative_result['normalized_score'], 'Sarah', 'o'),
    (moderate_result['normalized_score'], 'Michael', 's'),
    (aggressive_result['normalized_score'], 'Alex', '^')
]

for score, name, marker in investor_scores:
    ax.scatter(score, 0, s=200, marker=marker, color='black', zorder=5)
    ax.annotate(name, (score, 0.35), ha='center', fontsize=11, fontweight='bold')

ax.set_xlim(0, 100)
ax.set_ylim(-0.5, 1)
ax.set_xlabel('Risk Score', fontsize=12)
ax.set_title('Risk Profile Distribution', fontsize=14, fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)
ax.set_yticks([])

plt.tight_layout()
plt.show()

## Section 4: Asset Class Returns

Now let's generate synthetic historical returns for different asset classes. In production, robo-advisors use real historical data from providers like Bloomberg, Refinitiv, or free sources like Yahoo Finance.

**Just run this cell -- no programming knowledge needed!** The code below creates sample
investment data for eight different asset types (US stocks, international stocks, bonds, real
estate). In real life, robo-advisors pull this data from actual stock market prices. We use
simulated data so the notebook runs without needing an internet connection or a data subscription.

In [ ]:
@dataclass
class AssetClass:
    """Represents an asset class with expected return and risk."""
    name: str
    ticker: str  # Representative ETF
    expected_annual_return: float  # Expected return (decimal)
    annual_volatility: float  # Standard deviation (decimal)
    category: str  # 'equity', 'fixed_income', 'alternative'

class AssetDataGenerator:
    """
    Generate synthetic asset class returns for simulation.
    
    Uses realistic parameters based on historical averages.
    """
    
    def __init__(self):
        # Define asset classes with realistic parameters
        self.asset_classes = {
            'US_LARGE_CAP': AssetClass(
                name='US Large Cap Stocks',
                ticker='VTI',
                expected_annual_return=0.10,
                annual_volatility=0.16,
                category='equity'
            ),
            'US_SMALL_CAP': AssetClass(
                name='US Small Cap Stocks',
                ticker='VB',
                expected_annual_return=0.12,
                annual_volatility=0.20,
                category='equity'
            ),
            'INTL_DEVELOPED': AssetClass(
                name='International Developed',
                ticker='VEA',
                expected_annual_return=0.08,
                annual_volatility=0.17,
                category='equity'
            ),
            'EMERGING_MARKETS': AssetClass(
                name='Emerging Markets',
                ticker='VWO',
                expected_annual_return=0.11,
                annual_volatility=0.23,
                category='equity'
            ),
            'US_BONDS': AssetClass(
                name='US Aggregate Bonds',
                ticker='BND',
                expected_annual_return=0.04,
                annual_volatility=0.05,
                category='fixed_income'
            ),
            'TIPS': AssetClass(
                name='Treasury Inflation Protected',
                ticker='VTIP',
                expected_annual_return=0.03,
                annual_volatility=0.06,
                category='fixed_income'
            ),
            'INTL_BONDS': AssetClass(
                name='International Bonds',
                ticker='BNDX',
                expected_annual_return=0.03,
                annual_volatility=0.07,
                category='fixed_income'
            ),
            'REAL_ESTATE': AssetClass(
                name='Real Estate (REITs)',
                ticker='VNQ',
                expected_annual_return=0.09,
                annual_volatility=0.18,
                category='alternative'
            ),
        }
        
        # Correlation matrix (simplified)
        self._create_correlation_matrix()
    
    def _create_correlation_matrix(self):
        """Create realistic correlation matrix between assets."""
        # Asset order: US_LC, US_SC, INTL_DEV, EM, US_BONDS, TIPS, INTL_BONDS, REITS
        self.correlations = np.array([
            [1.00, 0.85, 0.75, 0.70, -0.10, 0.00, -0.05, 0.60],  # US Large Cap
            [0.85, 1.00, 0.70, 0.65, -0.15, -0.05, -0.10, 0.65],  # US Small Cap
            [0.75, 0.70, 1.00, 0.80, 0.00, 0.05, 0.10, 0.55],    # Intl Developed
            [0.70, 0.65, 0.80, 1.00, -0.05, 0.00, 0.05, 0.50],   # Emerging Markets
            [-0.10, -0.15, 0.00, -0.05, 1.00, 0.80, 0.60, 0.10], # US Bonds
            [0.00, -0.05, 0.05, 0.00, 0.80, 1.00, 0.50, 0.15],   # TIPS
            [-0.05, -0.10, 0.10, 0.05, 0.60, 0.50, 1.00, 0.10],  # Intl Bonds
            [0.60, 0.65, 0.55, 0.50, 0.10, 0.15, 0.10, 1.00],    # REITs
        ])
        self.asset_names = list(self.asset_classes.keys())
    
    def generate_returns(self, years: int = 10, frequency: str = 'monthly') -> Dict:
        """
        Generate correlated synthetic returns for all asset classes.
        
        Args:
            years: Number of years of data to generate
            frequency: 'daily', 'monthly', or 'annual'
            
        Returns:
            Dictionary with returns array and metadata
        """
        if frequency == 'daily':
            periods_per_year = 252
        elif frequency == 'monthly':
            periods_per_year = 12
        else:
            periods_per_year = 1
        
        n_periods = years * periods_per_year
        n_assets = len(self.asset_classes)
        
        # Get expected returns and volatilities (annualized)
        annual_returns = np.array([a.expected_annual_return for a in self.asset_classes.values()])
        annual_vols = np.array([a.annual_volatility for a in self.asset_classes.values()])
        
        # Convert to periodic
        periodic_returns = annual_returns / periods_per_year
        periodic_vols = annual_vols / np.sqrt(periods_per_year)
        
        # Create covariance matrix
        cov_matrix = np.outer(periodic_vols, periodic_vols) * self.correlations
        
        # Generate correlated random returns using Cholesky decomposition
        L = np.linalg.cholesky(cov_matrix)
        random_normals = np.random.randn(n_periods, n_assets)
        correlated_shocks = random_normals @ L.T
        
        # Add expected returns to get final returns
        returns = periodic_returns + correlated_shocks
        
        return {
            'returns': returns,
            'asset_names': self.asset_names,
            'n_periods': n_periods,
            'frequency': frequency,
            'years': years,
            'expected_returns': annual_returns,
            'volatilities': annual_vols,
            'correlations': self.correlations
        }
    
    def display_asset_summary(self):
        """Display summary of all asset classes."""
        print("\n" + "="*80)
        print("ASSET CLASS UNIVERSE")
        print("="*80)
        print(f"\n{'Asset Class':<30} {'ETF':<8} {'Exp. Return':<15} {'Volatility':<12} {'Category'}")
        print("-"*80)
        
        for key, asset in self.asset_classes.items():
            print(f"{asset.name:<30} {asset.ticker:<8} {asset.expected_annual_return*100:>8.1f}%      "
                  f"{asset.annual_volatility*100:>8.1f}%     {asset.category}")

# Create and display asset data
data_gen = AssetDataGenerator()
data_gen.display_asset_summary()

**What just happened?** The code above used something called a *Cholesky decomposition*
to generate realistic fake investment data. You do not need to understand the math -- the key
idea is simple: in real markets, stock prices tend to move somewhat together (when US stocks
go up, international stocks often go up too). The Cholesky technique ensures our simulated
data captures these realistic co-movements, so the results you see below behave like real markets.

> **Source note:** The correlation values and expected returns used here are based on
> approximate historical averages from US market data (2000--2024). Real robo-advisors
> update these estimates regularly using live market data feeds.

In [ ]:
# Generate 10 years of monthly returns
market_data = data_gen.generate_returns(years=10, frequency='monthly')

print(f"\nGenerated {market_data['n_periods']} periods of returns")
print(f"Shape: {market_data['returns'].shape} (periods x assets)")

# Calculate cumulative returns
cumulative_returns = np.cumprod(1 + market_data['returns'], axis=0)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot cumulative returns
ax1 = axes[0]
months = np.arange(market_data['n_periods'])
colors = plt.cm.tab10(np.linspace(0, 1, len(market_data['asset_names'])))

for i, (name, color) in enumerate(zip(market_data['asset_names'], colors)):
    asset = data_gen.asset_classes[name]
    ax1.plot(months/12, cumulative_returns[:, i], label=asset.name, color=color, alpha=0.8)

ax1.set_xlabel('Years')
ax1.set_ylabel('Growth of $1')
ax1.set_title('Simulated Asset Class Performance (10 Years)', fontweight='bold')
ax1.legend(loc='upper left', fontsize=8)
ax1.axhline(y=1, color='black', linestyle='--', alpha=0.3)

# Plot correlation heatmap
ax2 = axes[1]
im = ax2.imshow(market_data['correlations'], cmap='RdYlGn', vmin=-1, vmax=1)

# Add labels
short_names = ['US LC', 'US SC', 'Intl Dev', 'EM', 'US Bond', 'TIPS', 'Intl Bond', 'REITs']
ax2.set_xticks(range(len(short_names)))
ax2.set_yticks(range(len(short_names)))
ax2.set_xticklabels(short_names, rotation=45, ha='right')
ax2.set_yticklabels(short_names)
ax2.set_title('Asset Correlation Matrix', fontweight='bold')

# Add correlation values
for i in range(len(short_names)):
    for j in range(len(short_names)):
        val = market_data['correlations'][i, j]
        color = 'white' if abs(val) > 0.5 else 'black'
        ax2.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=8)

plt.colorbar(im, ax=ax2, label='Correlation')
plt.tight_layout()
plt.show()

## Section 5: Mean-Variance Optimization (Markowitz)

Modern Portfolio Theory (MPT), developed by Harry Markowitz in 1952, forms the
mathematical foundation of robo-advisors. The key insight: **diversification reduces
risk without necessarily reducing returns**.

### The Intuition (No Math Required)

A robo-advisor finds the best portfolio by:

1. **Looking at how each investment performed historically** -- some grow faster but
   swing wildly (stocks), while others grow slowly but steadily (bonds).
2. **Checking how investments move relative to each other** -- if one zigs when another
   zags, combining them smooths out the bumps. This is the power of diversification.
3. **Finding the mix that gives the best expected return for your comfort level with
   risk** -- there is no single "best" portfolio; it depends on how much volatility you
   can tolerate.

The algorithm searches through thousands of possible investment mixes and keeps only the
ones on the **efficient frontier** -- portfolios where you cannot get more return without
taking on more risk.

> **Technical aside (optional):** The underlying math uses matrix algebra to compute
> portfolio risk from the covariance between assets, then solves an optimization problem
> to find the best weights. The code does this for you automatically.

**You do NOT need to understand the code below.** Just run the cell and observe the
results. The code implements what we described above: finding the best mix of investments
that balances risk and return. It tries thousands of combinations and picks the ones that
sit on the efficient frontier.

In [ ]:
class MeanVarianceOptimizer:
    """
    Markowitz Mean-Variance Portfolio Optimization.
    
    Implements the classic portfolio optimization algorithm to find
    optimal asset allocations on the efficient frontier.
    """
    
    def __init__(self, expected_returns: np.ndarray, covariance_matrix: np.ndarray,
                 asset_names: List[str]):
        """
        Initialize optimizer with asset parameters.
        
        Args:
            expected_returns: Annual expected returns for each asset
            covariance_matrix: Covariance matrix of returns
            asset_names: Names of assets
        """
        self.returns = expected_returns
        self.cov_matrix = covariance_matrix
        self.asset_names = asset_names
        self.n_assets = len(expected_returns)
        
    def portfolio_return(self, weights: np.ndarray) -> float:
        """Calculate portfolio expected return."""
        return np.dot(weights, self.returns)
    
    def portfolio_volatility(self, weights: np.ndarray) -> float:
        """Calculate portfolio volatility (standard deviation)."""
        variance = np.dot(weights.T, np.dot(self.cov_matrix, weights))
        return np.sqrt(variance)
    
    def portfolio_sharpe_ratio(self, weights: np.ndarray, risk_free_rate: float = 0.02) -> float:
        """Calculate Sharpe ratio (risk-adjusted return)."""
        ret = self.portfolio_return(weights)
        vol = self.portfolio_volatility(weights)
        return (ret - risk_free_rate) / vol if vol > 0 else 0
    
    def random_portfolio(self) -> np.ndarray:
        """Generate random portfolio weights summing to 1."""
        weights = np.random.random(self.n_assets)
        return weights / weights.sum()
    
    def simulate_portfolios(self, n_portfolios: int = 10000) -> Dict:
        """
        Simulate random portfolios to visualize the opportunity set.
        
        Args:
            n_portfolios: Number of random portfolios to generate
            
        Returns:
            Dictionary with returns, volatilities, sharpe ratios, and weights
        """
        results = {
            'returns': [],
            'volatilities': [],
            'sharpe_ratios': [],
            'weights': []
        }
        
        for _ in range(n_portfolios):
            weights = self.random_portfolio()
            results['returns'].append(self.portfolio_return(weights))
            results['volatilities'].append(self.portfolio_volatility(weights))
            results['sharpe_ratios'].append(self.portfolio_sharpe_ratio(weights))
            results['weights'].append(weights)
        
        for key in ['returns', 'volatilities', 'sharpe_ratios']:
            results[key] = np.array(results[key])
        results['weights'] = np.array(results['weights'])
        
        return results
    
    def optimize_min_volatility(self, target_return: Optional[float] = None,
                                allow_short: bool = False) -> Dict:
        """
        Find the minimum volatility portfolio.
        
        Uses numerical optimization (gradient descent) for simplicity.
        Production systems would use quadratic programming (cvxpy, scipy.optimize).
        
        Args:
            target_return: If specified, find min vol portfolio at this return level
            allow_short: Whether to allow short selling (negative weights)
            
        Returns:
            Dictionary with optimal weights and portfolio metrics
        """
        # Start with equal weights
        weights = np.ones(self.n_assets) / self.n_assets
        
        learning_rate = 0.01
        n_iterations = 5000
        
        for _ in range(n_iterations):
            # Calculate gradient of volatility with respect to weights
            # d(sigma)/dw = Sigma * w / sigma
            vol = self.portfolio_volatility(weights)
            grad = np.dot(self.cov_matrix, weights) / vol if vol > 0 else np.zeros(self.n_assets)
            
            # Update weights
            weights = weights - learning_rate * grad
            
            # Project back to simplex (weights sum to 1, non-negative if no shorting)
            if not allow_short:
                weights = np.maximum(weights, 0)
            weights = weights / weights.sum()
        
        return {
            'weights': weights,
            'return': self.portfolio_return(weights),
            'volatility': self.portfolio_volatility(weights),
            'sharpe_ratio': self.portfolio_sharpe_ratio(weights)
        }
    
    def optimize_max_sharpe(self, risk_free_rate: float = 0.02,
                           allow_short: bool = False) -> Dict:
        """
        Find the maximum Sharpe ratio portfolio (tangency portfolio).
        
        Uses numerical optimization.
        """
        best_sharpe = -np.inf
        best_weights = None
        
        # Run multiple random starts to avoid local minima
        for _ in range(100):
            weights = self.random_portfolio()
            
            for _ in range(500):
                vol = self.portfolio_volatility(weights)
                ret = self.portfolio_return(weights)
                sharpe = (ret - risk_free_rate) / vol if vol > 0 else 0
                
                # Gradient of Sharpe ratio
                # d(SR)/dw = (mu - rf*1) / sigma - (ret - rf) * Sigma * w / sigma^3
                grad_ret = self.returns
                grad_vol = np.dot(self.cov_matrix, weights) / vol if vol > 0 else np.zeros(self.n_assets)
                
                grad_sharpe = grad_ret / vol - (ret - risk_free_rate) * grad_vol / (vol ** 2)
                
                # Gradient ascent (maximizing Sharpe)
                weights = weights + 0.01 * grad_sharpe
                
                if not allow_short:
                    weights = np.maximum(weights, 0)
                weights = weights / weights.sum()
            
            final_sharpe = self.portfolio_sharpe_ratio(weights, risk_free_rate)
            if final_sharpe > best_sharpe:
                best_sharpe = final_sharpe
                best_weights = weights.copy()
        
        return {
            'weights': best_weights,
            'return': self.portfolio_return(best_weights),
            'volatility': self.portfolio_volatility(best_weights),
            'sharpe_ratio': best_sharpe
        }
    
    def efficient_frontier(self, n_points: int = 50) -> Dict:
        """
        Calculate the efficient frontier.
        
        Args:
            n_points: Number of points on the frontier
            
        Returns:
            Dictionary with returns, volatilities, and weights for frontier portfolios
        """
        # Find return range
        min_ret = min(self.returns)
        max_ret = max(self.returns)
        target_returns = np.linspace(min_ret, max_ret, n_points)
        
        frontier = {
            'returns': [],
            'volatilities': [],
            'weights': []
        }
        
        # For each target return, find minimum volatility portfolio
        # Using simple grid search for educational purposes
        for target in target_returns:
            best_vol = np.inf
            best_weights = None
            
            # Generate many portfolios and find lowest vol near target return
            for _ in range(2000):
                w = self.random_portfolio()
                ret = self.portfolio_return(w)
                vol = self.portfolio_volatility(w)
                
                # Check if close to target return and lower volatility
                if abs(ret - target) < 0.005 and vol < best_vol:
                    best_vol = vol
                    best_weights = w
            
            if best_weights is not None:
                frontier['returns'].append(self.portfolio_return(best_weights))
                frontier['volatilities'].append(best_vol)
                frontier['weights'].append(best_weights)
        
        for key in ['returns', 'volatilities']:
            frontier[key] = np.array(frontier[key])
        
        return frontier

# Create covariance matrix from correlation and volatilities
vols = market_data['volatilities']
corr = market_data['correlations']
cov_matrix = np.outer(vols, vols) * corr

# Create optimizer
optimizer = MeanVarianceOptimizer(
    expected_returns=market_data['expected_returns'],
    covariance_matrix=cov_matrix,
    asset_names=market_data['asset_names']
)

print("Mean-Variance Optimizer initialized!")
print(f"\nNumber of assets: {optimizer.n_assets}")
print(f"Expected returns range: {min(optimizer.returns)*100:.1f}% to {max(optimizer.returns)*100:.1f}%")

> **Note:** The optimization above uses a simplified technique called *gradient descent*
> for educational purposes. Real robo-advisors use sophisticated optimization libraries
> (such as CVXPY or scipy.optimize) that are faster and more precise. The results here
> are illustrative -- the concepts are the same, but production systems are more robust.

In [ ]:
# Find key portfolios
print("\n" + "="*70)
print("PORTFOLIO OPTIMIZATION RESULTS")
print("="*70)

# Minimum volatility portfolio
min_vol_portfolio = optimizer.optimize_min_volatility()
print("\nMINIMUM VOLATILITY PORTFOLIO:")
print(f"  Expected Return:  {min_vol_portfolio['return']*100:.2f}%")
print(f"  Volatility:       {min_vol_portfolio['volatility']*100:.2f}%")
print(f"  Sharpe Ratio:     {min_vol_portfolio['sharpe_ratio']:.3f}")
print("\n  Allocation:")
for name, weight in zip(optimizer.asset_names, min_vol_portfolio['weights']):
    if weight > 0.01:  # Only show allocations > 1%
        asset = data_gen.asset_classes[name]
        print(f"    {asset.name:<30} {weight*100:>6.1f}%")

# Maximum Sharpe ratio portfolio
max_sharpe_portfolio = optimizer.optimize_max_sharpe()
print("\nMAXIMUM SHARPE RATIO PORTFOLIO:")
print(f"  Expected Return:  {max_sharpe_portfolio['return']*100:.2f}%")
print(f"  Volatility:       {max_sharpe_portfolio['volatility']*100:.2f}%")
print(f"  Sharpe Ratio:     {max_sharpe_portfolio['sharpe_ratio']:.3f}")
print("\n  Allocation:")
for name, weight in zip(optimizer.asset_names, max_sharpe_portfolio['weights']):
    if weight > 0.01:
        asset = data_gen.asset_classes[name]
        print(f"    {asset.name:<30} {weight*100:>6.1f}%")

## Section 6: Efficient Frontier Visualization

The efficient frontier shows all optimal portfolios - portfolios that maximize return for a given level of risk, or minimize risk for a given return.

Let's see the results visually. Look at how different risk levels lead to different investment mixes.

In [ ]:
# Simulate random portfolios and calculate efficient frontier
print("Simulating 10,000 random portfolios...")
simulated = optimizer.simulate_portfolios(n_portfolios=10000)

print("Calculating efficient frontier...")
frontier = optimizer.efficient_frontier(n_points=50)

# Create visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Plot simulated portfolios
scatter = ax.scatter(
    simulated['volatilities'] * 100,
    simulated['returns'] * 100,
    c=simulated['sharpe_ratios'],
    cmap='viridis',
    alpha=0.5,
    s=10
)

# Plot efficient frontier
# Sort frontier by volatility for proper line
sorted_idx = np.argsort(frontier['volatilities'])
ax.plot(
    frontier['volatilities'][sorted_idx] * 100,
    frontier['returns'][sorted_idx] * 100,
    'r-',
    linewidth=3,
    label='Efficient Frontier'
)

# Plot individual assets
for i, name in enumerate(optimizer.asset_names):
    asset = data_gen.asset_classes[name]
    ax.scatter(
        asset.annual_volatility * 100,
        asset.expected_annual_return * 100,
        s=150,
        marker='D',
        edgecolors='black',
        linewidths=2,
        zorder=5
    )
    ax.annotate(
        asset.ticker,
        (asset.annual_volatility * 100, asset.expected_annual_return * 100),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=9,
        fontweight='bold'
    )

# Plot key portfolios
ax.scatter(
    min_vol_portfolio['volatility'] * 100,
    min_vol_portfolio['return'] * 100,
    s=300,
    marker='*',
    color='blue',
    edgecolors='black',
    linewidths=2,
    zorder=10,
    label='Min Volatility'
)

ax.scatter(
    max_sharpe_portfolio['volatility'] * 100,
    max_sharpe_portfolio['return'] * 100,
    s=300,
    marker='*',
    color='gold',
    edgecolors='black',
    linewidths=2,
    zorder=10,
    label='Max Sharpe'
)

# Plot Capital Market Line (CML)
rf = 0.02  # Risk-free rate
max_vol = max(simulated['volatilities']) * 100
cml_vols = np.linspace(0, max_vol, 100)
cml_returns = rf * 100 + (max_sharpe_portfolio['sharpe_ratio']) * cml_vols
ax.plot(cml_vols, cml_returns, 'k--', linewidth=1.5, alpha=0.7, label='Capital Market Line')

# Formatting
ax.set_xlabel('Annual Volatility (%)', fontsize=12)
ax.set_ylabel('Expected Annual Return (%)', fontsize=12)
ax.set_title('Efficient Frontier and Optimal Portfolios', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.set_xlim(0, max_vol + 2)
ax.set_ylim(0, 15)

plt.colorbar(scatter, label='Sharpe Ratio')
plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("  The efficient frontier shows the best possible risk-return tradeoffs.")
print("  Portfolios BELOW the frontier are suboptimal (same risk, lower return).")
print("  Portfolios ABOVE the frontier are impossible to achieve.")

## Section 7: Automated Rebalancing Strategies

Over time, portfolios drift from their target allocations as some assets outperform others. **Rebalancing** brings the portfolio back to target weights. This is a core function of robo-advisors.

### Common Rebalancing Strategies

1. **Calendar Rebalancing**: Rebalance on fixed schedule (monthly, quarterly, annually)
2. **Threshold Rebalancing**: Rebalance when any asset drifts beyond a threshold (e.g., 5%)
3. **Hybrid**: Combine both approaches

**Just run this cell -- no programming knowledge needed!** The code below sets up
the rebalancing simulation engine. It tracks how a portfolio drifts over time and
tests different strategies for bringing it back on target.

In [ ]:
class Portfolio:
    """
    A portfolio that can be rebalanced using different strategies.
    """
    
    def __init__(self, initial_value: float, target_weights: np.ndarray,
                 asset_names: List[str]):
        """
        Initialize portfolio.
        
        Args:
            initial_value: Starting portfolio value
            target_weights: Target allocation weights
            asset_names: Names of assets
        """
        self.initial_value = initial_value
        self.target_weights = target_weights
        self.asset_names = asset_names
        self.n_assets = len(target_weights)
        
        # Initialize holdings at target weights
        self.values = initial_value * target_weights
        
        # Track history
        self.value_history = [initial_value]
        self.weight_history = [target_weights.copy()]
        self.rebalance_events = []
        
    @property
    def total_value(self) -> float:
        """Current total portfolio value."""
        return self.values.sum()
    
    @property
    def current_weights(self) -> np.ndarray:
        """Current portfolio weights."""
        return self.values / self.total_value
    
    def apply_returns(self, returns: np.ndarray) -> None:
        """Apply period returns to holdings."""
        self.values = self.values * (1 + returns)
        self.value_history.append(self.total_value)
        self.weight_history.append(self.current_weights.copy())
    
    def drift_from_target(self) -> np.ndarray:
        """Calculate drift of each asset from target weight."""
        return self.current_weights - self.target_weights
    
    def max_drift(self) -> float:
        """Maximum absolute drift from target."""
        return np.max(np.abs(self.drift_from_target()))
    
    def rebalance(self, period: int, reason: str = 'manual') -> Dict:
        """
        Rebalance portfolio back to target weights.
        
        Args:
            period: Current time period
            reason: Why rebalancing occurred
            
        Returns:
            Dictionary with rebalancing details
        """
        old_weights = self.current_weights.copy()
        total = self.total_value
        
        # Calculate trades needed
        target_values = total * self.target_weights
        trades = target_values - self.values
        
        # Execute rebalance
        self.values = target_values
        
        # Record event
        event = {
            'period': period,
            'reason': reason,
            'old_weights': old_weights,
            'new_weights': self.current_weights.copy(),
            'trades': trades,
            'turnover': np.sum(np.abs(trades)) / (2 * total)
        }
        self.rebalance_events.append(event)
        
        return event


class RebalancingSimulator:
    """
    Simulate different rebalancing strategies.
    """
    
    def __init__(self, returns: np.ndarray, asset_names: List[str]):
        """
        Args:
            returns: Array of shape (n_periods, n_assets) with returns
            asset_names: Names of assets
        """
        self.returns = returns
        self.asset_names = asset_names
        self.n_periods = returns.shape[0]
    
    def simulate_no_rebalancing(self, initial_value: float,
                                target_weights: np.ndarray) -> Portfolio:
        """Simulate buy-and-hold (no rebalancing)."""
        portfolio = Portfolio(initial_value, target_weights, self.asset_names)
        
        for period in range(self.n_periods):
            portfolio.apply_returns(self.returns[period])
        
        return portfolio
    
    def simulate_calendar_rebalancing(self, initial_value: float,
                                      target_weights: np.ndarray,
                                      frequency: int = 12) -> Portfolio:
        """
        Simulate calendar-based rebalancing.
        
        Args:
            frequency: Rebalance every N periods (12 = annual for monthly data)
        """
        portfolio = Portfolio(initial_value, target_weights, self.asset_names)
        
        for period in range(self.n_periods):
            portfolio.apply_returns(self.returns[period])
            
            # Rebalance on schedule
            if (period + 1) % frequency == 0:
                portfolio.rebalance(period, f'calendar ({frequency} periods)')
        
        return portfolio
    
    def simulate_threshold_rebalancing(self, initial_value: float,
                                       target_weights: np.ndarray,
                                       threshold: float = 0.05) -> Portfolio:
        """
        Simulate threshold-based rebalancing.
        
        Args:
            threshold: Rebalance when any asset drifts more than this (e.g., 0.05 = 5%)
        """
        portfolio = Portfolio(initial_value, target_weights, self.asset_names)
        
        for period in range(self.n_periods):
            portfolio.apply_returns(self.returns[period])
            
            # Rebalance if threshold exceeded
            if portfolio.max_drift() > threshold:
                portfolio.rebalance(period, f'threshold ({threshold*100:.0f}%)')
        
        return portfolio

# Create simulator with our market data
simulator = RebalancingSimulator(market_data['returns'], market_data['asset_names'])

print("Rebalancing Simulator initialized!")
print(f"Simulation period: {simulator.n_periods} months ({simulator.n_periods/12:.1f} years)")

In [ ]:
# Compare rebalancing strategies
initial_investment = 100000  # $100,000
target = max_sharpe_portfolio['weights']  # Use optimal portfolio

# Run simulations
no_rebal = simulator.simulate_no_rebalancing(initial_investment, target)
annual_rebal = simulator.simulate_calendar_rebalancing(initial_investment, target, frequency=12)
quarterly_rebal = simulator.simulate_calendar_rebalancing(initial_investment, target, frequency=3)
threshold_5pct = simulator.simulate_threshold_rebalancing(initial_investment, target, threshold=0.05)
threshold_10pct = simulator.simulate_threshold_rebalancing(initial_investment, target, threshold=0.10)

# Compare results
strategies = [
    ('No Rebalancing', no_rebal),
    ('Annual', annual_rebal),
    ('Quarterly', quarterly_rebal),
    ('5% Threshold', threshold_5pct),
    ('10% Threshold', threshold_10pct)
]

print("\n" + "="*80)
print("REBALANCING STRATEGY COMPARISON")
print("="*80)
print(f"\nInitial Investment: ${initial_investment:,.0f}")
print(f"Simulation Period: {simulator.n_periods/12:.0f} years\n")

print(f"{'Strategy':<20} {'Final Value':>15} {'Return':>12} {'# Rebalances':>15} {'Avg Turnover':>15}")
print("-"*80)

for name, portfolio in strategies:
    total_return = (portfolio.total_value / initial_investment - 1) * 100
    n_rebal = len(portfolio.rebalance_events)
    avg_turnover = np.mean([e['turnover'] for e in portfolio.rebalance_events]) * 100 if n_rebal > 0 else 0
    
    print(f"{name:<20} ${portfolio.total_value:>13,.0f} {total_return:>10.1f}% {n_rebal:>15} {avg_turnover:>13.1f}%")

In [ ]:
# Visualize portfolio drift and rebalancing
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Portfolio values over time
ax1 = axes[0, 0]
months = np.arange(len(no_rebal.value_history))
for name, portfolio in strategies:
    ax1.plot(months/12, portfolio.value_history, label=name, alpha=0.8)
ax1.set_xlabel('Years')
ax1.set_ylabel('Portfolio Value ($)')
ax1.set_title('Portfolio Growth Comparison', fontweight='bold')
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Weight drift for no-rebalancing portfolio
ax2 = axes[0, 1]
weight_history = np.array(no_rebal.weight_history)
for i, name in enumerate(no_rebal.asset_names):
    asset = data_gen.asset_classes[name]
    ax2.plot(months/12, weight_history[:, i] * 100, label=asset.name, alpha=0.8)
ax2.axhline(y=target[0]*100, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Years')
ax2.set_ylabel('Portfolio Weight (%)')
ax2.set_title('Weight Drift (No Rebalancing)', fontweight='bold')
ax2.legend(loc='upper right', fontsize=8)

# Weight stability for threshold rebalancing
ax3 = axes[1, 0]
weight_history = np.array(threshold_5pct.weight_history)
for i, name in enumerate(threshold_5pct.asset_names):
    asset = data_gen.asset_classes[name]
    ax3.plot(months/12, weight_history[:, i] * 100, label=asset.name, alpha=0.8)
# Mark rebalancing events
for event in threshold_5pct.rebalance_events:
    ax3.axvline(x=event['period']/12, color='red', alpha=0.3, linewidth=0.5)
ax3.set_xlabel('Years')
ax3.set_ylabel('Portfolio Weight (%)')
ax3.set_title('Weight Control (5% Threshold Rebalancing)', fontweight='bold')
ax3.legend(loc='upper right', fontsize=8)

# Rebalancing frequency comparison
ax4 = axes[1, 1]
strategy_names = [name for name, _ in strategies[1:]]  # Exclude no-rebal
rebal_counts = [len(p.rebalance_events) for _, p in strategies[1:]]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
bars = ax4.bar(strategy_names, rebal_counts, color=colors)
ax4.set_ylabel('Number of Rebalances')
ax4.set_title('Rebalancing Frequency by Strategy', fontweight='bold')
for bar, count in zip(bars, rebal_counts):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Takeaways:")
print("  1. Without rebalancing, portfolios drift significantly from targets")
print("  2. Threshold rebalancing trades only when necessary")
print("  3. More frequent rebalancing = higher transaction costs")
print("  4. Optimal strategy depends on trading costs and risk tolerance")

## Section 8: Tax-Loss Harvesting Basics

**Tax-loss harvesting** is a strategy to reduce taxes by selling investments at a loss to offset capital gains. This is a key differentiator for robo-advisors.

### How It Works

1. Identify holdings with unrealized losses
2. Sell to realize the loss (tax deduction)
3. Immediately buy a similar (but not "substantially identical") asset
4. Maintain portfolio exposure while capturing tax benefit

> **Note:** Tax rules vary by country. The examples below use US tax rules for
> illustration (including the "wash sale" rule). Your country may have different
> rules for investment gains and losses. Consult a local tax advisor for specifics.

### The Wash Sale Rule

You cannot buy a "substantially identical" security within 30 days before or after the sale, or the loss is disallowed.

**Just run this cell -- no programming knowledge needed!** The code below simulates
how robo-advisors automatically harvest tax losses. Focus on the output tables and
charts rather than the code itself.

In [ ]:
class TaxLossHarvester:
    """
    Tax-loss harvesting simulation.
    
    Demonstrates how robo-advisors capture tax losses while
    maintaining portfolio exposure.
    """
    
    def __init__(self, tax_rate: float = 0.25, wash_sale_period: int = 30):
        """
        Args:
            tax_rate: Marginal tax rate for capital gains
            wash_sale_period: Days before repurchase allowed (SEC rule: 30)
        """
        self.tax_rate = tax_rate
        self.wash_sale_period = wash_sale_period
        
        # Define substitute ETFs (similar exposure, not "substantially identical")
        self.substitutes = {
            'VTI': 'ITOT',   # Total US Market
            'VB': 'IJR',     # Small Cap
            'VEA': 'IEFA',   # International Developed
            'VWO': 'IEMG',   # Emerging Markets
            'BND': 'AGG',    # US Bonds
            'VTIP': 'TIP',   # TIPS
            'BNDX': 'IAGG',  # International Bonds
            'VNQ': 'IYR',    # REITs
        }
    
    def identify_harvesting_opportunities(self, holdings: Dict[str, Dict]) -> List[Dict]:
        """
        Identify positions with unrealized losses that can be harvested.
        
        Args:
            holdings: Dict with ticker -> {shares, cost_basis, current_price}
            
        Returns:
            List of harvesting opportunities
        """
        opportunities = []
        
        for ticker, position in holdings.items():
            cost_basis = position['shares'] * position['cost_basis']
            current_value = position['shares'] * position['current_price']
            unrealized_gain_loss = current_value - cost_basis
            
            if unrealized_gain_loss < 0:  # Loss position
                tax_benefit = abs(unrealized_gain_loss) * self.tax_rate
                
                opportunities.append({
                    'ticker': ticker,
                    'substitute': self.substitutes.get(ticker, 'CASH'),
                    'shares': position['shares'],
                    'cost_basis': cost_basis,
                    'current_value': current_value,
                    'unrealized_loss': unrealized_gain_loss,
                    'tax_benefit': tax_benefit
                })
        
        # Sort by tax benefit (largest first)
        opportunities.sort(key=lambda x: x['tax_benefit'], reverse=True)
        
        return opportunities
    
    def simulate_harvesting(self, initial_holdings: Dict[str, Dict],
                           price_changes: List[Dict]) -> Dict:
        """
        Simulate tax-loss harvesting over time.
        
        Args:
            initial_holdings: Starting portfolio
            price_changes: List of price change dicts per period
            
        Returns:
            Simulation results
        """
        holdings = {k: v.copy() for k, v in initial_holdings.items()}
        total_tax_saved = 0
        harvest_events = []
        
        for period, changes in enumerate(price_changes):
            # Update prices
            for ticker in holdings:
                if ticker in changes:
                    holdings[ticker]['current_price'] *= (1 + changes[ticker])
            
            # Check for harvesting opportunities
            opportunities = self.identify_harvesting_opportunities(holdings)
            
            # Harvest losses above threshold
            for opp in opportunities:
                if opp['tax_benefit'] > 100:  # Minimum $100 tax benefit
                    total_tax_saved += opp['tax_benefit']
                    harvest_events.append({
                        'period': period,
                        'ticker': opp['ticker'],
                        'substitute': opp['substitute'],
                        'loss_harvested': opp['unrealized_loss'],
                        'tax_benefit': opp['tax_benefit']
                    })
                    
                    # Reset cost basis (simplified - in reality would switch to substitute)
                    holdings[opp['ticker']]['cost_basis'] = holdings[opp['ticker']]['current_price']
        
        return {
            'total_tax_saved': total_tax_saved,
            'harvest_events': harvest_events,
            'final_holdings': holdings
        }

# Create sample portfolio
sample_holdings = {
    'VTI': {'shares': 100, 'cost_basis': 220.00, 'current_price': 215.00},
    'VB': {'shares': 50, 'cost_basis': 180.00, 'current_price': 170.00},
    'VEA': {'shares': 80, 'cost_basis': 45.00, 'current_price': 48.00},
    'BND': {'shares': 200, 'cost_basis': 82.00, 'current_price': 78.00},
    'VNQ': {'shares': 40, 'cost_basis': 95.00, 'current_price': 88.00},
}

harvester = TaxLossHarvester(tax_rate=0.25)
opportunities = harvester.identify_harvesting_opportunities(sample_holdings)

print("\n" + "="*70)
print("TAX-LOSS HARVESTING OPPORTUNITIES")
print("="*70)
print(f"\nTax Rate: {harvester.tax_rate*100:.0f}%\n")

print(f"{'Ticker':<8} {'Substitute':<10} {'Cost Basis':>12} {'Current':>12} {'Loss':>12} {'Tax Benefit':>12}")
print("-"*70)

total_benefit = 0
for opp in opportunities:
    print(f"{opp['ticker']:<8} {opp['substitute']:<10} "
          f"${opp['cost_basis']:>10,.0f} ${opp['current_value']:>10,.0f} "
          f"${opp['unrealized_loss']:>10,.0f} ${opp['tax_benefit']:>10,.0f}")
    total_benefit += opp['tax_benefit']

print("-"*70)
print(f"{'TOTAL TAX SAVINGS':>54} ${total_benefit:>10,.0f}")

print("\nHow it works:")
print("  1. Sell VB at loss of $500 -> Tax benefit of $125")
print("  2. Immediately buy IJR (similar small-cap exposure)")
print("  3. Portfolio exposure unchanged, but captured tax loss")
print("  4. After 30 days, can switch back to VB if desired")

In [ ]:
# Visualize tax-loss harvesting impact over time
np.random.seed(42)

# Generate price changes for simulation
n_months = 36
price_changes = []
for _ in range(n_months):
    changes = {}
    for ticker in sample_holdings:
        changes[ticker] = np.random.normal(0.005, 0.03)  # Monthly returns
    price_changes.append(changes)

# Simulate
results = harvester.simulate_harvesting(sample_holdings, price_changes)

print("\n" + "="*70)
print("TAX-LOSS HARVESTING SIMULATION (3 Years)")
print("="*70)

print(f"\nTotal Tax Savings: ${results['total_tax_saved']:,.0f}")
print(f"Number of Harvest Events: {len(results['harvest_events'])}")

if results['harvest_events']:
    print(f"\nHarvest Events:")
    for event in results['harvest_events'][:10]:  # Show first 10
        print(f"  Month {event['period']:2d}: Sold {event['ticker']} -> {event['substitute']}, "
              f"Loss: ${event['loss_harvested']:,.0f}, Tax Benefit: ${event['tax_benefit']:,.0f}")

# Calculate cumulative tax savings
if results['harvest_events']:
    cumulative_savings = np.zeros(n_months + 1)
    for event in results['harvest_events']:
        cumulative_savings[event['period']:] += event['tax_benefit']
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.fill_between(range(n_months + 1), cumulative_savings, alpha=0.3, color='green')
    ax.plot(range(n_months + 1), cumulative_savings, 'g-', linewidth=2)
    ax.set_xlabel('Months')
    ax.set_ylabel('Cumulative Tax Savings ($)')
    ax.set_title('Tax-Loss Harvesting: Cumulative Tax Savings Over Time', fontweight='bold')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    plt.tight_layout()
    plt.show()

## Section 9: Robo vs Traditional Advisors

Let's compare robo-advisors with traditional human advisors across various dimensions.

The comparison below is printed as formatted text. Scan the **SERVICES** table to
see where robo-advisors shine versus where human advisors still have the edge.

In [ ]:
def compare_robo_vs_traditional():
    """
    Display comprehensive comparison of robo vs traditional advisors.
    """
    comparison = """
    ROBO-ADVISOR vs TRADITIONAL ADVISOR COMPARISON
    =============================================
    
    FEES
    ----
    Robo-Advisor:       0.15% - 0.50% of AUM per year
    Traditional:        1.00% - 2.00% of AUM per year
    
    On $100,000 portfolio over 30 years (7% return):
      Robo (0.25%):     Final value = $706,000  |  Fees paid = $55,000
      Traditional (1%): Final value = $574,000  |  Fees paid = $188,000
      DIFFERENCE:       +$132,000 with robo-advisor
    
    MINIMUM INVESTMENT
    ------------------
    Robo-Advisor:       $0 - $5,000
    Traditional:        $100,000 - $1,000,000+
    
    SERVICES
    --------
    | Feature                   | Robo | Traditional |
    |---------------------------|------|-------------|
    | Portfolio Management      |  Yes |     Yes     |
    | Automatic Rebalancing     |  Yes |   Usually   |
    | Tax-Loss Harvesting       |  Yes |   Sometimes |
    | Financial Planning        | Basic|  Extensive  |
    | Estate Planning           |  No  |     Yes     |
    | Insurance Advice          |  No  |     Yes     |
    | Business Planning         |  No  |     Yes     |
    | Complex Tax Strategies    |  No  |     Yes     |
    | Behavioral Coaching       |  No  |     Yes     |
    | 24/7 Access               |  Yes |     No      |
    | Emotional Support         |  No  |     Yes     |
    
    BEST FOR
    --------
    Robo-Advisor:
      - Beginners with limited capital
      - Hands-off investors
      - Simple financial situations
      - Cost-conscious investors
      - Tech-savvy users
    
    Traditional Advisor:
      - High-net-worth individuals
      - Complex financial situations
      - Business owners
      - Those needing comprehensive planning
      - Investors who value personal relationships
    
    HYBRID MODELS
    -------------
    Many providers now offer hybrid solutions:
    - Vanguard Personal Advisor Services (0.30%)
    - Betterment Premium (0.40%)
    - Schwab Intelligent Portfolios Premium ($300/year + fee)
    
    These combine algorithmic management with human advisor access.
    """
    print(comparison)

compare_robo_vs_traditional()

In [ ]:
# Visualize fee impact over time
def simulate_fee_impact(initial_investment: float, annual_return: float,
                       years: int, fee_rates: Dict[str, float]) -> Dict:
    """
    Simulate portfolio growth with different fee structures.
    """
    results = {}
    
    for name, fee in fee_rates.items():
        values = [initial_investment]
        fees_paid = [0]
        
        for year in range(years):
            # Grow portfolio
            value = values[-1] * (1 + annual_return)
            # Deduct fees
            fee_amount = value * fee
            value -= fee_amount
            
            values.append(value)
            fees_paid.append(fees_paid[-1] + fee_amount)
        
        results[name] = {
            'values': values,
            'fees_paid': fees_paid,
            'final_value': values[-1],
            'total_fees': fees_paid[-1]
        }
    
    return results

# Simulate
fee_rates = {
    'DIY (0%)': 0.00,
    'Robo (0.25%)': 0.0025,
    'Hybrid (0.50%)': 0.005,
    'Traditional (1.00%)': 0.01,
    'High-Fee (1.50%)': 0.015,
}

fee_simulation = simulate_fee_impact(
    initial_investment=100000,
    annual_return=0.07,
    years=30,
    fee_rates=fee_rates
)

# Print summary
print("\n" + "="*70)
print("FEE IMPACT ANALYSIS ($100,000 over 30 years at 7% return)")
print("="*70)

print(f"\n{'Fee Structure':<25} {'Final Value':>15} {'Fees Paid':>15} {'Lost to Fees':>15}")
print("-"*70)

baseline = fee_simulation['DIY (0%)']['final_value']
for name, data in fee_simulation.items():
    lost_to_fees = baseline - data['final_value']
    print(f"{name:<25} ${data['final_value']:>13,.0f} ${data['total_fees']:>13,.0f} ${lost_to_fees:>13,.0f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Portfolio growth
ax1 = axes[0]
colors = ['#27ae60', '#3498db', '#9b59b6', '#e74c3c', '#c0392b']
for (name, data), color in zip(fee_simulation.items(), colors):
    ax1.plot(range(31), data['values'], label=name, color=color, linewidth=2)
ax1.set_xlabel('Years')
ax1.set_ylabel('Portfolio Value ($)')
ax1.set_title('Portfolio Growth by Fee Structure', fontweight='bold')
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Final values bar chart
ax2 = axes[1]
names = list(fee_simulation.keys())
final_values = [data['final_value'] for data in fee_simulation.values()]
total_fees = [data['total_fees'] for data in fee_simulation.values()]

x = np.arange(len(names))
width = 0.35

bars1 = ax2.bar(x - width/2, final_values, width, label='Final Value', color='#3498db')
bars2 = ax2.bar(x + width/2, total_fees, width, label='Total Fees Paid', color='#e74c3c')

ax2.set_xlabel('Fee Structure')
ax2.set_ylabel('Amount ($)')
ax2.set_title('Final Value vs Total Fees Paid', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([n.split('(')[0].strip() for n in names], rotation=45, ha='right')
ax2.legend()
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

print("\nKey Insight:")
robo_value = fee_simulation['Robo (0.25%)']['final_value']
trad_value = fee_simulation['Traditional (1.00%)']['final_value']
print(f"  Over 30 years, a robo-advisor saves ${(robo_value - trad_value):,.0f} compared to traditional!")
print(f"  That's {((robo_value - trad_value) / trad_value * 100):.1f}% more money in your pocket.")

## Challenges

### Discussion Challenges (No coding required)
1. **Risk Profile Analysis**: Look at the risk profiles above. Why might a young investor choose a different profile than a retiree? What factors should matter?
2. **Robo vs. Human**: What are three advantages and three disadvantages of using a robo-advisor vs. a human financial advisor?
3. **Bias Check**: Could a robo-advisor discriminate against certain groups of people? How?

### Hands-On Challenges (Just change numbers and re-run -- no coding knowledge needed!)
The cells below let you experiment by changing parameter values. Try the suggestions
in each cell and observe how the results change.

In [ ]:
# CHALLENGE 1: Create Your Own Risk Profile
# ==========================================
# Fill in YOUR answers to the risk questionnaire

print("\n" + "="*70)
print("CHALLENGE 1: Create Your Risk Profile")
print("="*70)

# YOUR TURN: Enter your responses (0-4 for each question)
# See Section 3 for the questions
your_responses = [
    2,  # Q1: Investment time horizon (0=<1yr, 4=>15yr)
    2,  # Q2: Reaction to 20% drop (0=sell all, 4=buy more)
    2,  # Q3: Investment goal (0=preserve capital, 4=aggressive growth)
    2,  # Q4: Experience (0=none, 4=extensive)
    2,  # Q5: % of savings (0=>75%, 4=<10%)
    3,  # Q6: Income stability (0=unstable, 4=very stable)
]

your_profile = profiler.calculate_risk_score(your_responses)

print(f"\nYour Risk Profile:")
print(f"  Score: {your_profile['normalized_score']:.1f}/100")
print(f"  Classification: {your_profile['profile']}")
print(f"  {your_profile['description']}")

# TRY THIS: Change the numbers above and re-run the cell.
# What happens if you change Q1 from 2 to 4 (longer time horizon)?
# What happens if you change Q2 from 2 to 0 (more risk-averse)?
# How does each answer shift your risk profile?

In [ ]:
# CHALLENGE 2: Build a Custom Portfolio
# =====================================
# TRY THIS: Change the weight numbers below and re-run.
# The weights must add up to 1.0 (100%).
# What happens if you put 0.60 in US Large Cap and reduce bonds to 0.05?
# What happens if you put 0.50 in US Bonds and split the rest evenly?

print("\n" + "="*70)
print("CHALLENGE 2: Build Your Portfolio")
print("="*70)

# Change these weights and re-run. They must sum to 1.0.
# Asset order: US_LC, US_SC, INTL_DEV, EM, US_BONDS, TIPS, INTL_BONDS, REITS
your_weights = np.array([
    0.30,  # US Large Cap (VTI)
    0.10,  # US Small Cap (VB)
    0.15,  # International Developed (VEA)
    0.05,  # Emerging Markets (VWO)
    0.25,  # US Bonds (BND)
    0.05,  # TIPS (VTIP)
    0.05,  # International Bonds (BNDX)
    0.05,  # REITs (VNQ)
])

# Validate weights sum to 1
assert abs(your_weights.sum() - 1.0) < 0.001, "Weights must sum to 1.0!"

# Calculate portfolio metrics
your_return = optimizer.portfolio_return(your_weights)
your_vol = optimizer.portfolio_volatility(your_weights)
your_sharpe = optimizer.portfolio_sharpe_ratio(your_weights)

print(f"\nYour Portfolio Metrics:")
print(f"  Expected Return:  {your_return*100:.2f}%")
print(f"  Volatility:       {your_vol*100:.2f}%")
print(f"  Sharpe Ratio:     {your_sharpe:.3f}")

print(f"\nComparison with Optimal Portfolios:")
print(f"  Min Vol Sharpe:   {min_vol_portfolio['sharpe_ratio']:.3f}")
print(f"  Max Sharpe:       {max_sharpe_portfolio['sharpe_ratio']:.3f}")
print(f"  Your Sharpe:      {your_sharpe:.3f}")

# Visualize your allocation
fig, ax = plt.subplots(figsize=(10, 6))
labels = [data_gen.asset_classes[n].name for n in optimizer.asset_names]
colors = plt.cm.tab10(np.linspace(0, 1, len(labels)))

# Filter out zero allocations
mask = your_weights > 0.01
filtered_weights = your_weights[mask]
filtered_labels = [l for l, m in zip(labels, mask) if m]
filtered_colors = colors[mask]

wedges, texts, autotexts = ax.pie(
    filtered_weights,
    labels=filtered_labels,
    autopct='%1.1f%%',
    colors=filtered_colors,
    startangle=90
)
ax.set_title('Your Portfolio Allocation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# CHALLENGE 3: Simulate Your Portfolio Over Time
# ==============================================

print("\n" + "="*70)
print("CHALLENGE 3: Simulate Your Portfolio")
print("="*70)

# Simulate with different rebalancing strategies
your_initial = 50000  # $50,000 starting capital

your_no_rebal = simulator.simulate_no_rebalancing(your_initial, your_weights)
your_annual = simulator.simulate_calendar_rebalancing(your_initial, your_weights, frequency=12)
your_threshold = simulator.simulate_threshold_rebalancing(your_initial, your_weights, threshold=0.05)

# Compare
your_strategies = [
    ('No Rebalancing', your_no_rebal),
    ('Annual Rebalancing', your_annual),
    ('5% Threshold', your_threshold)
]

print(f"\nStarting Capital: ${your_initial:,.0f}")
print(f"Simulation Period: {simulator.n_periods/12:.0f} years\n")

for name, portfolio in your_strategies:
    total_return = (portfolio.total_value / your_initial - 1) * 100
    print(f"{name:<20}: ${portfolio.total_value:>10,.0f} ({total_return:>+6.1f}%)")

# TRY THIS: Change your_initial from 50000 to 100000 or 10000.
# Does the starting amount affect which strategy wins?
# Go back to Challenge 2, change your weights, then re-run this cell.

In [ ]:
# CHALLENGE 4: Experiment with Hybrid Rebalancing
# ================================================
# TRY THIS: Change the calendar_frequency and threshold values below.
# What happens if you rebalance monthly (frequency=1) instead of quarterly (3)?
# What happens if you raise the threshold from 0.05 to 0.10?

print("\n" + "="*70)
print("CHALLENGE 4: Custom Hybrid Rebalancing Strategy")
print("="*70)

def simulate_hybrid_rebalancing(simulator, initial_value: float,
                                target_weights: np.ndarray,
                                calendar_frequency: int = 3,
                                threshold: float = 0.05) -> Portfolio:
    """
    YOUR TURN: Implement hybrid rebalancing.
    
    Rebalance when EITHER:
    1. Calendar trigger (every calendar_frequency periods)
    2. Threshold trigger (any asset drifts more than threshold)
    
    Returns:
        Portfolio with hybrid rebalancing applied
    """
    portfolio = Portfolio(initial_value, target_weights, simulator.asset_names)
    
    for period in range(simulator.n_periods):
        portfolio.apply_returns(simulator.returns[period])
        
        # The hybrid logic: rebalance if EITHER condition is met
        calendar_trigger = (period + 1) % calendar_frequency == 0
        threshold_trigger = portfolio.max_drift() > threshold
        
        if calendar_trigger or threshold_trigger:
            reason = 'calendar' if calendar_trigger else 'threshold'
            portfolio.rebalance(period, f'hybrid ({reason})')
    
    return portfolio

# Test your implementation
hybrid_portfolio = simulate_hybrid_rebalancing(
    simulator,
    initial_value=100000,
    target_weights=max_sharpe_portfolio['weights'],
    calendar_frequency=3,  # Quarterly
    threshold=0.05  # 5%
)

print(f"\nHybrid Strategy Results:")
print(f"  Final Value: ${hybrid_portfolio.total_value:,.0f}")
print(f"  Number of Rebalances: {len(hybrid_portfolio.rebalance_events)}")

# Count rebalance reasons
calendar_count = sum(1 for e in hybrid_portfolio.rebalance_events if 'calendar' in e['reason'])
threshold_count = sum(1 for e in hybrid_portfolio.rebalance_events if 'threshold' in e['reason'])
print(f"    - Calendar triggers: {calendar_count}")
print(f"    - Threshold triggers: {threshold_count}")

## Summary

In this notebook, you built a complete robo-advisor simulation from scratch and learned:

### Key Concepts

1. **Robo-Advisor Mechanics**
   - Automated investment platforms using algorithms
   - Low-cost alternative to traditional advisors
   - Core features: risk profiling, portfolio optimization, rebalancing, tax harvesting

2. **Risk Profiling**
   - Questionnaire-based assessment
   - Maps investor characteristics to risk tolerance scores
   - Drives portfolio allocation decisions

3. **Mean-Variance Optimization (Markowitz)**
   - Portfolio return: weighted sum of asset returns
   - Portfolio risk: depends on covariance between assets
   - Efficient frontier: best risk-return tradeoffs
   - Sharpe ratio: risk-adjusted return measure

4. **Automated Rebalancing**
   - Calendar-based: rebalance on fixed schedule
   - Threshold-based: rebalance when drift exceeds limit
   - Tradeoff between staying on target and transaction costs

5. **Tax-Loss Harvesting**
   - Sell losing positions to capture tax deductions
   - Replace with similar assets to maintain exposure
   - Wash sale rule: 30-day waiting period

6. **Robo vs Traditional Advisors**
   - Robo: low fees, accessible, automated, limited services
   - Traditional: comprehensive planning, personal relationship, higher fees
   - Fee impact compounds significantly over time

### Real-World Implementation Notes

Production robo-advisors add:
- Real market data integration (APIs)
- Regulatory compliance (SEC, FINRA)
- More sophisticated optimization (Black-Litterman, risk parity)
- Factor-based investing
- ESG/sustainable investing options
- Goal-based planning (retirement, house, education)

### Further Reading

- [Markowitz, H. (1952) "Portfolio Selection"](https://www.jstor.org/stable/2975974)
- [Betterment's Tax-Loss Harvesting White Paper](https://www.betterment.com/resources/tax-loss-harvesting)
- [Vanguard's Principles for Investing Success](https://www.vanguard.com/pdf/ISGPRINC.pdf)
- [CFA Institute: Robo-Advisors Research](https://www.cfainstitute.org/research/industry-research/robo-advisors)